In [13]:
!pip install -U sentence-transformers
!pip install rank_bm25
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import requests
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from rank_bm25 import BM25Okapi
import time

In [7]:
# Updated API Endpoint for Semantic Scholar
SEMANTIC_SCHOLAR_BASE_URL = "https://api.semanticscholar.org/graph/v1/paper/search"
HEADERS = {
    "User-Agent": "DeepCite/1.0",
}

# Function to fetch papers from Semantic Scholar API based on a query
def fetch_papers_by_keywords(keywords, fields="title,abstract,url,year,citationCount,authors", limit=1000):
    papers = []
    offset = 0
    batch_size = 100  # Smaller batch size to avoid rate limiting

    while len(papers) < limit:
        params = {
            "query": keywords,
            "fields": fields,
            "limit": min(batch_size, limit - len(papers)),
            "offset": offset
        }

        try:
            response = requests.get(SEMANTIC_SCHOLAR_BASE_URL, headers=HEADERS, params=params)

            if response.status_code == 200:
                data = response.json()
                new_papers = data.get("data", [])

                # Break if no new papers or fewer papers than requested (end of results)
                if not new_papers:
                    break

                papers.extend(new_papers)
                offset += len(new_papers)

                # Add delay between requests to avoid rate limiting
                time.sleep(1)

            elif response.status_code == 429:
                print(f"Rate limit exceeded. Waiting 30 seconds before retry...")
                time.sleep(30)  # Wait longer if rate limited
                continue
            else:
                print(f"Failed to fetch data, status code: {response.status_code}")
                break

        except Exception as e:
            print(f"Error fetching papers: {str(e)}")
            break

        # Check if we've reached all available papers
        if len(papers) >= offset:
            break

    return papers[:limit]

In [10]:
# Initialize the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def get_embeddings(papers):
    """Generate embeddings for the list of papers using SentenceTransformer."""
    if not papers:
        return None

    titles_and_abstracts = []
    for paper in papers:
        title = paper.get('title', '')  # Default to empty string if title is missing
        abstract = paper.get('abstract', '')  # Default to empty string if abstract is missing
        title = title if title else ''  # Ensure title is a string
        abstract = abstract if abstract else ''  # Ensure abstract is a string
        titles_and_abstracts.append(title + " " + abstract)

    embeddings = model.encode(titles_and_abstracts, convert_to_tensor=True)
    return embeddings

In [11]:
# Function to create and store embeddings in FAISS index
def create_faiss_index(embeddings):
    """Create and store embeddings in FAISS index."""
    if embeddings is None:
        return None

    # Convert embeddings to a NumPy array for FAISS
    embeddings_np = np.array(embeddings.cpu().detach().numpy()).astype('float32')

    # Create the FAISS index (using L2 distance, Euclidean)
    index = faiss.IndexFlatL2(embeddings_np.shape[1])
    index.add(embeddings_np)  # Add embeddings to the index
    return index

In [12]:
def get_bm25_scores(query, papers):
    """Compute BM25 similarity scores between query and papers."""
    if not papers:
        return []

    # Preprocess documents: tokenize title + abstract
    tokenized_corpus = [
        ((paper.get('title') or '') + ' ' + (paper.get('abstract') or '')).lower().split()
        for paper in papers]
    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)

    return scores

In [13]:
def search_query(query, faiss_index, papers, top_k=1000):
    """Search query in FAISS index and retrieve semantic + BM25 scores."""
    if not papers or faiss_index is None:
        return [], [], []

    # Limit top_k to the number of available papers
    top_k = min(top_k, len(papers))
    if top_k == 0:
        return [], [], []

    # FAISS
    query_embedding = model.encode([query], convert_to_tensor=True)
    query_embedding_np = np.array(query_embedding.cpu().detach().numpy()).astype('float32')
    distances, indices = faiss_index.search(query_embedding_np, top_k)

    # Make sure indices are unique to avoid duplicates
    unique_indices = []
    seen = set()
    for idx in indices[0]:
        if idx not in seen and idx < len(papers):
            seen.add(idx)
            unique_indices.append(idx)

    recommended_papers = [papers[i] for i in unique_indices]

    # Get corresponding distances for the unique indices
    faiss_dists = [distances[0][list(indices[0]).index(i)] for i in unique_indices]

    # BM25
    bm25_scores = get_bm25_scores(query, recommended_papers)

    return recommended_papers, faiss_dists, bm25_scores

In [14]:
def rank_by_weighted_score_hybrid(papers, faiss_dists, bm25_scores, weights=(0.5, 0.3, 0.2), hybrid_weights=(0.7, 0.3)):
    """Rank papers with hybrid relevance (FAISS + BM25) and citation/recency."""
    relevance_weight, citation_weight, recency_weight = weights
    w_faiss, w_bm25 = hybrid_weights

    # Normalize FAISS distances into similarity
    max_dist = max(faiss_dists) + 1e-5
    faiss_sims = [1 - (d / max_dist) for d in faiss_dists]

    # Normalize BM25 scores
    max_bm25 = max(bm25_scores) + 1e-5
    bm25_sims = [s / max_bm25 for s in bm25_scores]

    # Filter valid citation counts and years
    valid_citations = [p['citationCount'] for p in papers if p.get('citationCount') is not None]
    valid_years = [p['year'] for p in papers if p.get('year') is not None]

    if not valid_citations or not valid_years:
        raise ValueError("Missing valid 'citationCount' or 'year' values in papers.")

    max_citation = max(valid_citations) + 1e-5
    max_year = max(valid_years) + 1e-5
    min_year = min(valid_years)

    ranked = []
    for i, paper in enumerate(papers):
        combined_relevance = (w_faiss * faiss_sims[i]) + (w_bm25 * bm25_sims[i])

        citation = paper.get('citationCount')
        year = paper.get('year')

        norm_citation = (citation / max_citation) if citation is not None else 0
        norm_recency = ((year - min_year) / (max_year - min_year)) if year is not None else 0

        final_score = (
            relevance_weight * combined_relevance +
            citation_weight * norm_citation +
            recency_weight * norm_recency
        )

        ranked.append({
            "Title": paper['title'],
            "Abstract": paper.get('abstract', 'N/A'),
            "DOI": paper.get('url', "N/A"),
            "Citation Count": paper['citationCount'],
            "Year": paper['year'],
            "FAISS Similarity": round(faiss_sims[i], 4),
            "BM25 Similarity": round(bm25_sims[i], 4),
            "Combined Relevance": round(combined_relevance, 4),
            "Final Score": round(final_score, 4)
        })

    ranked.sort(key=lambda x: -x['Final Score'])
    return ranked

In [15]:
# Main function to fetch, rank, and return relevant papers based on a query
def get_ranked_papers(query, weights=(0.5, 0.3, 0.2), hybrid_weights=(0.7, 0.3), limit=1000):
    print(f"Searching for papers on: '{query}'")
    papers = fetch_papers_by_keywords(query, limit=limit)

    if not papers:
        return "No relevant papers found."

    print(f"Found relevant papers. Creating embeddings...")

    embeddings = get_embeddings(papers)
    faiss_index = create_faiss_index(embeddings)

    print("Searching and ranking papers...")
    recommended_papers, faiss_dists, bm25_scores = search_query(query, faiss_index, papers, top_k=len(papers))

    if not recommended_papers:
        return "No relevant papers could be ranked."

    ranked_papers = rank_by_weighted_score_hybrid(recommended_papers, faiss_dists, bm25_scores, weights, hybrid_weights)
    return ranked_papers

In [20]:
# Example usage
def run_query(query, limit=1000):
    ranked_papers = get_ranked_papers(query, limit=limit)


    print("\nTop Recommended Papers:\n")
    for i, paper in enumerate(ranked_papers[:1000], 1):
        print(f"{i}. {paper['Title']}")
        print(f"   DOI: {paper['DOI']}")
        print(f"   Citations: {paper['Citation Count']} | Year: {paper['Year']}")
        print(f"   Relevance Score: {round(paper['Final Score'] * 100, 2)}%")
        print(f"   Abstract: {paper['Abstract'] or 'N/A'}")
        print()

    return ranked_papers

In [18]:
len(ranked_papers)

100

In [21]:
query = "Supervised Machine Learning in Healthcare"
# Lower the limit to avoid rate limiting during testing
ranked_papers = run_query(query, limit=1000)

Searching for papers on: 'Supervised Machine Learning in Healthcare'
Found relevant papers. Creating embeddings...
Searching and ranking papers...

Top Recommended Papers:

1. A Survey on the Explainability of Supervised Machine Learning
   DOI: https://www.semanticscholar.org/paper/5fca8bbec714e403fa0f95a56b355c8ca835bcc0
   Citations: 722 | Year: 2020
   Relevance Score: 63.48%
   Abstract: Predictions obtained by, e.g., artificial neural networks have a high accuracy but humans often perceive the models as black boxes. Insights about the decision making are mostly opaque for humans. Particularly understanding the decision making in highly sensitive areas such as healthcare or finance, is of paramount importance. The decision-making behind the black boxes requires it to be more transparent, accountable, and understandable for humans. This survey paper provides essential definitions, an overview of the different principles and methodologies of explainable Supervised Machine Learning (